In [1]:
from datetime import datetime, timedelta
from tqdm import tqdm
from GQLib.Framework import Framework
from GQLib.Optimizers import MPGA, PSO, SGA, SA, NELDER_MEAD, TABU, FA
from GQLib.Models import LPPL, LPPLS
from GQLib.enums import InputType
from GQLib.AssetProcessor import AssetProcessor
import numpy as np

## Creation des données

In [21]:
fw = Framework(frequency="daily", input_type=InputType.BTC)

start_date = datetime(day=1, month=1, year=2015)
end_date = datetime(day=1, month=9, year=2024)

# Calculer le nombre total d'itérations
total_iterations = (end_date - start_date).days // 30 + 1

current_date = start_date

with tqdm(total=total_iterations) as pbar:
    while current_date <= end_date:
        fw_start_dt = (current_date - timedelta(days=365 * 3))
        fw_start = fw_start_dt.strftime("%d/%m/%Y")
        fw_end = current_date.strftime("%d/%m/%Y")

        results = fw.process(fw_start, fw_end, NELDER_MEAD(LPPLS))

        file_name = f"Strategy2/strat_{current_date.strftime('%m-%Y')}"
        fw.save_results(results, file_name)
        best_results = fw.analyze(results, lppl_model=LPPLS)

        # Incrémenter la date et mettre à jour la barre de progression
        current_date += timedelta(days=30)
        pbar.update(1)

  0%|          | 0/118 [00:00<?, ?it/s]

100%|██████████| 118/118 [47:49<00:00, 24.32s/it]


In [18]:
import plotly.graph_objects as go

# Créer un graphique interactif avec Plotly
fig = go.Figure()

# Ajouter les données de prix
fig.add_trace(go.Scatter(x=fw.global_dates, y=fw.global_prices, mode='lines', name='Prix'))

# Ajouter un titre
fig.update_layout(
    title='Évolution des prix',
    xaxis_title='Date',
    yaxis_title='Prix'  # ou 'plotly', 'seaborn', etc. pour personnaliser le style
)

# Afficher le graphique
fig.show()


## Strat

In [ ]:
start_date = datetime(day=1, month=2, year=2015)
file_name = f"Strategy2/strat_{current_date.strftime('%m-%Y')}"
best_results = fw.analyze(result_json_name=file_name,significativity_tc=0.3, lppl_model=LPPLS)

In [19]:
fw = Framework(frequency="weekly", input_type=InputType.BTC)

start_date = datetime(day=1, month=1, year=2015)
end_date = datetime(day=1, month=9, year=2024)

# Calculer le nombre total d'itérations
total_iterations = (end_date - start_date).days // 90 + 1

current_date = start_date
nb_tc = 10
with tqdm(total=total_iterations) as pbar:
    while current_date <= end_date:
        fw_start_dt = (current_date - timedelta(days=365 * 3))
        fw_start = fw_start_dt.strftime("%d/%m/%Y")
        fw_end = current_date.strftime("%d/%m/%Y")

        file_name = f"Strategy/strat_{current_date.strftime('%m-%Y')}"
        best_results = fw.analyze(result_json_name=file_name, significativity_tc=0.01)
        nb_true = len([x for x in best_results if x['is_significant']==True])/len(best_results)

        filtered_indices = [i for i, date in enumerate(fw.global_dates) if fw_start_dt <= date <= current_date]
        if not filtered_indices:
            print(f"Aucune donnée disponible entre {fw_start} et {fw_end}.")
            current_date += timedelta(days=90)
            pbar.update(1)
            continue

        filtered_dates = [fw.global_dates[i] for i in filtered_indices]
        filtered_prices = [fw.global_prices[i] for i in filtered_indices]
        min_time = np.inf
        max_time = -np.inf
        significant_tc = []
        for res in best_results:
            if res["sub_start"] < min_time:
                min_time = res["sub_start"]
            if res["sub_end"] > max_time:
                max_time = res["sub_end"]
            if res["is_significant"]:
                significant_tc.append([res["bestParams"][0], res["power_value"]])
            
            
        try:
            if (nb_tc != None):
                significant_tc = sorted(significant_tc, key=lambda x: x[1], reverse=True)[:min(len(significant_tc),nb_tc)]
            #Calcul de la date exacte du tc en pondérant nb_tc par leur power
            sum_max_power = sum(x[1] for x in significant_tc if x[1] is not None and not np.isnan(x[1]))
            weighted_sum_tc = sum(x[0] * x[1] for x in significant_tc if x[1] is not None and not np.isnan(x[1]))
            significant_tc = weighted_sum_tc / sum_max_power if sum_max_power != 0 else 0
        except Exception as e:
            print(f"Erreur lors du calcul des TC : {e}")
            significant_tc_value = None
        if significant_tc and isinstance(significant_tc, float):
            if len(fw.global_dates) > significant_tc + 1> 0:
                tc_date = fw.global_dates[int(round(significant_tc))]
                diff_date : datetime = tc_date-current_date
                if diff_date.days<100:
                    print(f"Date TC {tc_date} et diff date {diff_date}")
        
        # Incrémenter la date et mettre à jour la barre de progression
        current_date += timedelta(days=90)
        pbar.update(1)

  0%|          | 0/40 [00:00<?, ?it/s]

 10%|█         | 4/40 [00:05<00:52,  1.45s/it]

Date TC 2015-12-07 00:00:00 et diff date 70 days 00:00:00


 25%|██▌       | 10/40 [00:16<00:49,  1.66s/it]

Date TC 2017-04-03 00:00:00 et diff date 13 days 00:00:00


...

 95%|█████████▌| 38/40 [00:59<00:03,  1.51s/it]

Date TC 2024-03-18 00:00:00 et diff date 34 days 00:00:00


 98%|█████████▊| 39/40 [01:00<00:01,  1.53s/it]

Date TC 2024-05-27 00:00:00 et diff date 14 days 00:00:00


100%|██████████| 40/40 [01:02<00:00,  1.56s/it]


## SP500

In [22]:
fw = Framework(frequency="daily", input_type=InputType.SP500)
from dateutil.relativedelta import relativedelta
start_date = datetime(day=1, month=1, year=2010)
end_date = datetime(day=1, month=12, year=2024)

# Calculer le nombre total d'itérations
total_iterations = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1

current_date = start_date

with tqdm(total=total_iterations) as pbar:
    while current_date <= end_date:
        fw_start_dt = (current_date - relativedelta(years=3))
        fw_start = fw_start_dt.strftime("%d/%m/%Y")
        fw_end = current_date.strftime("%d/%m/%Y")

        results = fw.process(fw_start, fw_end, NELDER_MEAD(LPPLS))

        file_name = f"Strategy_sp/strat_{current_date.strftime('%m-%Y')}"
        fw.save_results(results, file_name)
        best_results = fw.analyze(results, lppl_model=LPPLS)

        # Incrémenter la date et mettre à jour la barre de progression
        current_date += relativedelta(months=1)
        pbar.update(1)

100%|██████████| 180/180 [1:05:57<00:00, 21.99s/it]


In [5]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
from tqdm import tqdm



from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np

# Initialisation du framework
fw = Framework(frequency="daily", input_type=InputType.SP500)

# Paramètres de la stratégie
start_date = datetime(day=1, month=1, year=2010)
end_date = datetime(day=1, month=12, year=2024)
total_iterations = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
nb_tc = 10
initial_capital = 1000  # Capital initial en USD
capital = initial_capital
position = 0  # Quantité d'actif détenue
short_position = 0  # Quantité engagée en short

holding = False  # Statut long
shorting = False  # Statut short
current_date = start_date

# Fonction pour récupérer le prix correspondant à une date
def get_price_at_date(current_date, global_dates, global_prices):
    # Vérifier si current_date est dans global_dates
    for i, date in enumerate(global_dates):
        if date >= current_date:  # Trouve la première date égale ou plus grande
            return global_prices[i]
    # Si la date est après la dernière date disponible
    return global_prices[-1]  # Retourner le dernier prix disponible

entry_price = None
# Boucle principale
with tqdm(total=total_iterations) as pbar:
    while current_date <= end_date:
        
        closing_price = get_price_at_date(current_date, fw.global_dates, fw.global_prices)
        if (entry_price is None):
            entry_price = closing_price

        # Définir les plages de dates
        fw_start_dt = current_date - relativedelta(years=3)
        fw_start = fw_start_dt.strftime("%d/%m/%Y")
        fw_end = current_date.strftime("%d/%m/%Y")

        # Analyse du framework
        file_name = f"Strategy_sp/strat_{current_date.strftime('%m-%Y')}"
        best_results = fw.analyze(result_json_name=file_name,significativity_tc=0.3, lppl_model=LPPLS)

        # Analyse des résultats significatifs
        significant_tc = []
        for res in best_results:
            if res["is_significant"]:
                significant_tc.append([res["bestParams"][0], res["power_value"]])

        # Calcul de la date TC pondérée par leur power
        significant_tc_value = None
        try:
            if significant_tc:
                significant_tc = sorted(significant_tc, key=lambda x: x[1], reverse=True)[:min(len(significant_tc), nb_tc)]
                sum_max_power = sum(x[1] for x in significant_tc if x[1] is not None and not np.isnan(x[1]))
                weighted_sum_tc = sum(x[0] * x[1] for x in significant_tc if x[1] is not None and not np.isnan(x[1]))
                significant_tc_value = weighted_sum_tc / sum_max_power if sum_max_power != 0 else None
        except Exception as e:
            print(f"Erreur lors du calcul des TC : {e}")

       
        # Validation et décision de stratégie
        if significant_tc_value is not None and isinstance(significant_tc_value, float):
            tc_index = int(round(significant_tc_value))
            if 0 <= tc_index < len(fw.global_dates):
                tc_date = fw.global_dates[tc_index]
                diff_date = tc_date - current_date
                tc_price = fw.global_prices[tc_index]
                # Short si diff_date.days < 100
                if diff_date.days < 30 and not shorting:
                    
                    if holding:
                        # Fermer position longue
                        rdt = (tc_price/entry_price)-1
                        capital*= (1+ rdt)
                        print(f"[{tc_date}] Long fermé à {tc_price}, gain = {rdt:.2f}, capital = {capital:.2f}")
                        holding = False
                        

                    # Ouvrir une position short
                    entry_price = tc_price  # Stocker le prix d'entrée pour le short
                    shorting = True
                    print(f"[{tc_date}] Short ouvert à {tc_price}")
                
                # Si diff_date.days >= 100, fermer le short
                elif shorting and diff_date.days >= 30:
                    rdt = (entry_price/closing_price)-1
                    capital*= (1+ rdt)
                    print(f"[{current_date}] Short fermé à {closing_price}, gain = {rdt:.2f}, capital = {capital:.2f}")
                    shorting = False
                    

        # Si aucune condition de short, ouvrir/maintenir une position longue
        if not shorting and not holding:
            
            holding = True
            entry_price = closing_price
            print(f"[{current_date}] Long ouvert à {entry_price}")

        # Avancer dans le temps et mettre à jour la barre de progression
        current_date += relativedelta(months=1)
        pbar.update(1)

closing_price = get_price_at_date(current_date, fw.global_dates, fw.global_prices)
# Clôturer toute position ouverte à la fin
if holding:
    rdt = (closing_price/entry_price)-1
    capital*= (1+ rdt)
    print(f"[{current_date}] Position longue clôturée à {closing_price}, capital final = {capital:.2f}")
elif shorting:
    rdt = (entry_price/closing_price)-1
    capital*= (1+ rdt)
    print(f"[{current_date}] Position short clôturée à {closing_price}, capital final = {capital:.2f}")

# Résultat final
gain = capital - initial_capital
print(f"Capital initial : {initial_capital:.2f}, Capital final : {capital:.2f}, Gain total : {gain:.2f} ({(gain / initial_capital) * 100:.2f}%)")


  1%|          | 1/180 [00:18<56:29, 18.93s/it]

[2010-01-01 00:00:00] Long ouvert à 1132.989990234375


 41%|████      | 73/180 [18:15<22:15, 12.48s/it]

[2015-12-23 00:00:00] Long fermé à 2064.2900390625, gain = 0.82, capital = 1821.98
[2015-12-23 00:00:00] Short ouvert à 2064.2900390625


 41%|████      | 74/180 [18:27<21:58, 12.44s/it]

...

[2020-07-24 00:00:00] Long fermé à 3215.6298828125, gain = 0.05, capital = 2530.07
[2020-07-24 00:00:00] Short ouvert à 3215.6298828125


 72%|███████▏  | 129/180 [29:50<10:22, 12.20s/it]

[2020-09-01 00:00:00] Short fermé à 3526.64990234375, gain = -0.09, capital = 2306.94
[2020-09-01 00:00:00] Long ouvert à 3526.64990234375


100%|██████████| 180/180 [41:34<00:00, 13.86s/it]

[2025-01-01 00:00:00] Position longue clôturée à 5868.5498046875, capital final = 3838.87
Capital initial : 1000.00, Capital final : 3838.87, Gain total : 2838.87 (283.89%)


## Run BTC

In [ ]:
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from dateutil.relativedelta import relativedelta
# Initialisation du framework
fw = Framework(frequency="daily", input_type=InputType.BTC)

# Paramètres de la stratégie
start_date = datetime(day=1, month=1, year=2015)
end_date = datetime(day=1, month=9, year=2024)

total_iterations = (end_date - start_date).days // 30 + 1
#total_iterations = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1

nb_tc = 10
initial_capital = 100  # Capital initial en USD
capital = initial_capital
capital_long = initial_capital

holding = False  # Statut long
shorting = False  # Statut short

current_date = start_date

# Listes pour stocker les résultats
capital_values = []
positions = []
dates = []
long_only_capital_values = []
# Fonction pour récupérer le prix correspondant à une date
def get_price_at_date(current_date, global_dates, global_prices):
    # Vérifier si current_date est dans global_dates
    for i, date in enumerate(global_dates):
        if date >= current_date:  # Trouve la première date égale ou plus grande
            return global_prices[i]
    # Si la date est après la dernière date disponible
    return global_prices[-1]  # Retourner le dernier prix disponible

entry_price = None
# Boucle principale
with tqdm(total=total_iterations) as pbar:
    
    while current_date <= end_date:
        closing_price = get_price_at_date(current_date, fw.global_dates, fw.global_prices)

        if current_date != start_date:
            old_price = get_price_at_date(current_date - timedelta(days=30), fw.global_dates, fw.global_prices)
            capital_long *= (closing_price / old_price)  # Mise à jour du capital long

        long_only_capital_values.append(capital_long)
        if holding:
            rdt = (closing_price / entry_price) - 1
            capital *= (1 + rdt)  # Mise à jour du capital pour position longue
            
        elif shorting:
            rdt = (entry_price / closing_price) - 1
            capital *= (1 + rdt)  # Mise à jour du capital pour position short

        entry_price = closing_price

        # Définir les plages de dates
        fw_start_dt = current_date - relativedelta(years=3)
        fw_start = fw_start_dt.strftime("%d/%m/%Y")
        fw_end = current_date.strftime("%d/%m/%Y")

        # Analyse du framework
        file_name = f"Strategy2/strat_{current_date.strftime('%m-%Y')}"
        best_results = fw.analyze(result_json_name=file_name,significativity_tc=0.3, lppl_model=LPPLS)

        # Analyse des résultats significatifs
        significant_tc = []
        for res in best_results:
            if res["is_significant"]:
                significant_tc.append([res["bestParams"][0], res["power_value"]])

        # Calcul de la date TC pondérée par leur power
        significant_tc_value = None
        try:
            if significant_tc:
                significant_tc = sorted(significant_tc, key=lambda x: x[1], reverse=True)[:min(len(significant_tc), nb_tc)]
                sum_max_power = sum(x[1] for x in significant_tc if x[1] is not None and not np.isnan(x[1]))
                weighted_sum_tc = sum(x[0] * x[1] for x in significant_tc if x[1] is not None and not np.isnan(x[1]))
                significant_tc_value = weighted_sum_tc / sum_max_power if sum_max_power != 0 else None
        except Exception as e:
            print(f"Erreur lors du calcul des TC : {e}")

        # Validation et décision de stratégie
        if significant_tc_value is not None and isinstance(significant_tc_value, float):
            tc_index = int(round(significant_tc_value))
            if 0 <= tc_index < len(fw.global_dates):
                tc_date = fw.global_dates[tc_index]
                diff_date = tc_date - current_date
                tc_price = fw.global_prices[tc_index]
                # Short si diff_date.days < 30
                if diff_date.days < 30 and not shorting:
                    
                    if holding:
                        # Fermer position longue
                        rdt = (tc_price/entry_price)-1
                        capital*= (1+ rdt)
                        print(f"[{tc_date}] Long fermé à {tc_price}, gain = {rdt:.2f}, capital = {capital:.2f}")
                        holding = False

                    # Ouvrir une position short
                    entry_price = tc_price  # Stocker le prix d'entrée pour le short
                    shorting = True
                    print(f"[{tc_date}] Short ouvert à {tc_price}")
                
                # Si diff_date.days >= 100, fermer le short
                elif shorting and diff_date.days >= 30:
                    # rdt = (entry_price/closing_price)-1
                    # capital*= (1+ rdt)
                    print(f"[{current_date}] Short fermé à {closing_price}, gain = {rdt:.2f}, capital = {capital:.2f}")
                    shorting = False
                

        # Si aucune condition de short, ouvrir/maintenir une position longue
        if not shorting and not holding:
            holding = True
            entry_price = closing_price
            print(f"[{current_date}] Long ouvert à {entry_price}")

        capital_values.append(capital)
        positions.append("Long" if holding else ("Short" if shorting else "None"))
        dates.append(current_date.strftime("%Y-%m-%d"))
        # Avancer dans le temps et mettre à jour la barre de progression
        current_date += timedelta(days=30)
        pbar.update(1)

closing_price = get_price_at_date(current_date, fw.global_dates, fw.global_prices)
old_price = get_price_at_date(current_date - timedelta(days=30), fw.global_dates, fw.global_prices)
capital_long *= (closing_price / old_price)  # Mise à jour du capital long
long_only_capital_values.append(capital_long)

# Clôturer toute position ouverte à la fin
if holding:
    rdt = (closing_price/entry_price)-1
    capital*= (1+ rdt)
    print(f"[{current_date}] Position longue clôturée à {closing_price}, capital final = {capital:.2f}")
elif shorting:
    rdt = (entry_price/closing_price)-1
    capital*= (1+ rdt)
    print(f"[{current_date}] Position short clôturée à {closing_price}, capital final = {capital:.2f}")
capital_values.append(capital)
# Résultat final
gain = capital - initial_capital
print(f"Capital initial : {initial_capital:.2f}, Capital final : {capital:.2f}, Gain total : {gain:.2f} ({(gain / initial_capital) * 100:.2f}%)")


  1%|          | 1/118 [00:03<07:32,  3.87s/it]

[2015-01-01 00:00:00] Long ouvert à 314.2489929199219


 10%|█         | 12/118 [01:42<19:58, 11.31s/it]